## 훈련알고리즘에 데이터 주입

미니배치 경사 하강법을 구현하기 위해 6.6.gradient_descent에서 사용한 코드를 변경해 구현<br/>
이렇게 하려면 매 반복에서 X와 y를 다음번 미니 배치로 바꿔야한다.<br/>
가장 간단한 방법으로는 `placeholder`노드를 사용하는것이다.<br/>
> placeholder 선언시 아무것도 하지 않고 그래프 실행할 때 데이터를 주입할 수 있는 변수 <br/>
> 실행시 값이 지정되지 않으면 에러가 발생..

In [1]:
import tensorflow as tf

In [5]:
# 플레이스홀더 노드 A와 B를 만든다.
# B를 평가할 때 eval() 메서드에 feed_dict 매개변수로 A의 값을 전달
# A는 랭크가 2(즉 2차원)이고 열은 3개여야한다.(그렇지 않으면 에러발생)
# 행의 개수는 상관없음

A = tf.placeholder(tf.float32, shape=(None, 3), name='A')
B = A + 5

In [6]:
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})
    
print(B_val_1)
print()
print(B_val_2)

[[6. 7. 8.]]

[[ 9. 10. 11.]
 [12. 13. 14.]]


> 실제로는 플레이스홀더뿐만 아니라 어떤 연산의 출력값도 주입할 수 있다. 이런 경우 텐서플로는 이 연산을 평가하지 않고 주입된 값을 사용함<br/>
> 신경망의 은닉층에 캐싱된 결과를 주입하기 위해 사용(동결된 층 캐싱하기)

In [13]:
# 값을 지정하지 않으면 에러 발생
# InvalidArgumentError (see above for traceback): You must feed a value for placeholder tensor 'A' with dtype float and shape [?,3]
#	 [[node A (defined at <ipython-input-5-79c71d038d20>:6) ]]


#with tf.Session() as sess:
    #B_val_1 = B.eval()

In [16]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf

In [31]:
# 데이터 셋을 추출하고 
# 모든 훈련 샘플에 편향에 대한 입력 특성(x0 = 1)을 추가
housing = fetch_california_housing()
m, n = housing.data.shape
print(m, n)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)

scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]   # (m, 1)행렬을 만든뒤 housing데이터의 값을 채워넣음.
scaled_housing_data_plus_bias

20640 8


array([[ 1.        ,  2.34476576,  0.98214266, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.        ,  1.7826994 ,  1.85618152, ..., -0.02584253,
         1.03850269, -1.33282653],
       ...,
       [ 1.        , -1.14259331, -0.92485123, ..., -0.0717345 ,
         1.77823747, -0.8237132 ],
       [ 1.        , -1.05458292, -0.84539315, ..., -0.09122515,
         1.77823747, -0.87362627],
       [ 1.        , -0.78012947, -1.00430931, ..., -0.04368215,
         1.75014627, -0.83369581]])

In [32]:
n_epochs = 1000
learning_rate = 0.01

# 이전 사용 코드
# X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")             # (20640, 9)
# y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')      # (20640, 1)

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')     # (9, 1)
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)  # theta 라는 변수에 아래 연산을 할당..

In [33]:
batch_size = 100                              # 배치 크기 
n_batch_size = int(np.ceil(m / batch_size))   # 전체 배치 횟수

In [35]:
def fetch_batch(epoch, batch_index, batch_size):
    # 디스크로부터 데이터 적재하기
    np.random.seed(epoch * n_batch_size + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batch_size):
            X_batch, Y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: Y_batch})
            
    best_theta = theta.eval()
    print(best_theta)

[[ 2.0684314e+00]
 [ 8.3556569e-01]
 [ 1.2917145e-01]
 [-2.5932044e-01]
 [ 2.9060593e-01]
 [-6.1495457e-04]
 [-4.0860951e-02]
 [-8.1783104e-01]
 [-7.8949809e-01]]
